# MEng-Team-Project-ML

This notebook contains the initial analysis of JSON files generated \
by our ML system as of December 2022 (refer to git commit for exact \
version).

Each one of our key objectives is potentially non-trivial to implement. \
The issues with each objective are listed below: 
- Object Identification 
  - This objective is the easiest to implement as we have the predicted \
    object along with it's bounding box per frame. However, across different \
    frames the object and camera can experience different scenarios which \
    can make relying on any individual frame invalid. For example, the \
    `00001.01350_2022-12-07T15-35-24.000Z.mp4` test file in the daytime \
    dataset on Google Drive shows the bus which is driving away from the \
    camera as being misidentified as a truck closer towards the middle / \
    end of the video. This is a common theme throughout the TFL JamCam \
    datasets. One way to rectify this, is to take the object predictions across \
    the entire time the object is in view (by taking it's predicted anchor value) \
    and determining the mode of the predicted classes. This is likely to be correct \
    for nice scenarios (day time, no glare). This needs to be deliberated further \
    for harder conditions such as night time / snow, etc.
- Object Count
  - For this objective, although it may appear simple on the surface, is \
    actually non-trivial because we have to correctly count the number of \
    objects across a time period. This becomes difficult for two reasons, \
    firstly we have to correctly identify the same object moving across \
    the field of view which can be problematic if the object is momentarily \
    obscured for whatever reason (e.g. camera blur, goes out of view and then \
    back into view. For example in our case, an object could cross from one exit \
    onto another exit in the Bunarby Road junction but be blocked by a HGV vehicle. \
    This would now count as a new object which would affect our counting of the objects). \
    The second issue is if the object is momentarily miscategorised which means that \
    yolov7's default method of tracking the object considers it a different object.
- Object Tracking
  - Similar to the above issue, object tracking relies on being able to identify the object \
    you are tracking as the same object across different frames. If an object is misidentified \
    partially before the video ends or the object exits the frame, this can cause the object \
    tracker to treat the object which was already being tracked as a new object. In theory, \
    we could implement code to see where a route / object was already being tracked and treat \
    it as the same object if it is reasonably close to what was being tracked before. This requires \
    testing / experimenting / tweaking. It is likely we can get a solution which is good enough \
    but I don't think we can straight up solve this issue as it has complex, non-trivial edge cases.

## Load Dataset

### Load COCO Classes

In [1]:
import yaml

COCO_YAML_PATH = "../yolov7-segmentation/data/coco.yaml"

def load_coco_classes(coco_settings):
    with open(coco_settings, "r") as stream:
        try:
            coco_data = yaml.safe_load(stream)
            coco_data = coco_data["names"]
            return coco_data
        except yaml.YAMLError as exc:
            print(exc)

COCO_CLASSES = load_coco_classes(COCO_YAML_PATH)

### Load Pre-Saved Analytical Dataset (SQLite)